# USB Wifi Example

In this notebook, a wifi dongle has been plugged into the Zybo board.  Specifically a RALink wifi dongle commonly used with Raspberry Pi kits is connected into the board.  Using Linux calls and Python functions, we will determine the unique name of the dongle and then create a network entry for a known ssid/password pair.  This demo was first done using an iPhone hotspot wireless connection.

References <br>
http://www.canakit.com/raspberry-pi-wifi.html <br>

#### 1. Need to find Wifi Device using Linux calls

In [151]:
network_devices_bash_str = !ip a
network_devices_bash_str.l

['1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default ',
 '    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00',
 '    inet 127.0.0.1/8 scope host lo',
 '       valid_lft forever preferred_lft forever',
 '    inet6 ::1/128 scope host ',
 '       valid_lft forever preferred_lft forever',
 '2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000',
 '    link/ether 00:1e:c0:dd:a1:bb brd ff:ff:ff:ff:ff:ff',
 '    inet 172.20.9.92/22 brd 172.20.11.255 scope global eth0',
 '       valid_lft forever preferred_lft forever',
 '    inet6 fe80::21e:c0ff:fedd:a1bb/64 scope link ',
 '       valid_lft forever preferred_lft forever',
 '3: wlx000f6002a692: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000',
 '    link/ether 00:0f:60:02:a6:92 brd ff:ff:ff:ff:ff:ff',
 '    inet 172.20.10.4/28 brd 172.20.10.15 scope global wlx000f6002a692',
 '       valid_lft forever preferred_lft forever'

#### 2. Use Python regular expressions to get wifi port name

In [152]:
#2. 
import re

for line in network_devices_bash_str.l:
    m = re.match('^([\d]): ([\w]+): *', line)
    if m:
        if m.group(2) != 'lo' and m.group(2) != 'eth0':
            wifi_port = m.group(2)

wifi_port

'wlx000f6002a692'

#### 3. Generate the correct /etc/network/interfaces.d entry

In [147]:
def gen_network_password_file(wifi_port,ssid,password):

    # get bash string into string format for key search
    wifikey_bash_str = !wpa_passphrase $ssid $password
    wifikey_tokens = str(wifikey_bash_str).replace("'",'').split(',')
    
    # search clean list for tpsk key value
    for key_val in wifikey_tokens:
        if '\\tpsk=' in key_val:
            wifi_wpa_key = key_val.split('=')[1]
    
    # write the network interface file with new ssid/password entry
    !ip link set $wifi_port up
    
    net_iface_fh = open("/etc/network/interfaces.d/" + wifi_port, 'w')
    net_iface_fh.write("iface " + wifi_port + " inet dhcp\n")
    net_iface_fh.write(" wpa-ssid " + ssid + "\n")
    net_iface_fh.write(" wpa-psk " +   wifi_wpa_key + "\n\n")
    net_iface_fh.close()
        
    !ifup {wifi_port}


#### 4. Call the above function with ssid/password for your wifi network

In [153]:
# Do custom ssid/password for your wifi connection - below is example from iPhone hotspot
gen_network_password_file(wifi_port,'iPhone','iphonehs')

ifup: interface wlx000f6002a692 already configured


#### 5. Verify Wifi network has IP address

In [150]:
!ifconfig

eth0      Link encap:Ethernet  HWaddr 00:1e:c0:dd:a1:bb  
          inet addr:172.20.9.92  Bcast:172.20.11.255  Mask:255.255.252.0
          inet6 addr: fe80::21e:c0ff:fedd:a1bb/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:73250 errors:106 dropped:0 overruns:102 frame:4
          TX packets:6176 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:11409389 (11.4 MB)  TX bytes:3900962 (3.9 MB)
          Interrupt:54 Base address:0xb000 

lo        Link encap:Local Loopback  
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
          UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:2475 errors:0 dropped:0 overruns:0 frame:0
          TX packets:2475 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0 
          RX bytes:943488 (943.4 KB)  TX bytes:943488 (943.4 KB)

wlx000f6002a692 Link encap:Ethernet  HWaddr 00:0f:60: